# Efficient Yelp API Calls - CORE
Author: Paul Foy

## Imports and Installation

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi
!pip install tqdm

In [3]:
# Load API Credentials
with open('/Users/pafoy_gz6qkdd/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
    
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

## Set API location and term

In [31]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'ITALIAN'

## Create JSON File

In [32]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_Italian.json"
JSON_FILE

'Data/results_in_progress_NY_Italian.json'

In [34]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_Italian.json already exists.


In [35]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [36]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [37]:
## How many results total?
total_results = results['total']
total_results

8500

In [38]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [39]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

425

In [40]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [41]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/425 [00:00<?, ?it/s]

In [43]:
# Define function to check if file exists
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [44]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_Italian.json not found. Saving empty list to new file.
- 0 previous results found.


425

In [45]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


  0%|          | 0/425 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


## Create new DF from JSON file

In [46]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,4749,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",+12122436546,(212) 243-6546,3186.380498
1,hdiuRS9sVZSMReZm4oV5SA,da-andrea-new-york,Da Andrea,https://s3-media1.fl.yelpcdn.com/bphoto/OciLdd...,False,https://www.yelp.com/biz/da-andrea-new-york?ad...,2028,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.736218, 'longitude': -73.99597}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '35 W 13th St', 'address2': '', '...",+12123671979,(212) 367-1979,3424.729337
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3123,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
3,OCTiJwvjoK81WoDwsTkFvA,paesano-of-mulberry-street-new-york,Paesano of Mulberry Street,https://s3-media2.fl.yelpcdn.com/bphoto/yN9ARC...,False,https://www.yelp.com/biz/paesano-of-mulberry-s...,1407,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.718554, 'longitude': -73.997447}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '136 Mulberry St', 'address2': ''...",+12129651188,(212) 965-1188,1481.706653
4,rp8S7scxOkVVeG7dNneMGQ,l-artusi-new-york,L'Artusi,https://s3-media2.fl.yelpcdn.com/bphoto/mf-sqE...,False,https://www.yelp.com/biz/l-artusi-new-york?adj...,2350,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.73372, 'longitude': -74.00513}","[pickup, delivery]",$$$,"{'address1': '228 W 10th St', 'address2': '', ...",+12122555757,(212) 255-5757,3272.999061


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,4_aoi1DOrrzRnyG-egxWYQ,parashades-pizza-and-cocktails-brooklyn,Parashades Pizza and Cocktails,https://s3-media3.fl.yelpcdn.com/bphoto/BV8enU...,False,https://www.yelp.com/biz/parashades-pizza-and-...,8,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 40.6810334, 'longitude': -73.9749...",[],NaN,"{'address1': '241 Flatbush Ave', 'address2': N...",+12124571168,(212) 457-1168,3166.991152
996,lQ8qDNaTPnewJt6xwG__gQ,leland-eating-and-drinking-house-brooklyn,Leland Eating and Drinking House,https://s3-media2.fl.yelpcdn.com/bphoto/BWtMaa...,False,https://www.yelp.com/biz/leland-eating-and-dri...,78,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.5,"{'latitude': 40.679822105618534, 'longitude': ...",[delivery],NaN,"{'address1': '755 Dean St', 'address2': '', 'a...",+16464707008,(646) 470-7008,3763.980228
997,Ucg3flg-2tJAahEePIoigQ,magdalene-brooklyn,Magdalene,https://s3-media2.fl.yelpcdn.com/bphoto/9ez5kN...,False,https://www.yelp.com/biz/magdalene-brooklyn?ad...,65,"[{'alias': 'lebanese', 'title': 'Lebanese'}, {...",4.0,"{'latitude': 40.71252, 'longitude': -73.94899}","[pickup, delivery]",$$,"{'address1': '524 Lorimer St', 'address2': '',...",+13475088280,(347) 508-8280,3902.389737
998,NBeQAgTo9hlJxgp9xbMO-w,tufino-pizzeria-napoletana-astoria-28,Tufino Pizzeria Napoletana,https://s3-media4.fl.yelpcdn.com/bphoto/S24th9...,False,https://www.yelp.com/biz/tufino-pizzeria-napol...,471,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.773986, 'longitude': -73.908126}","[pickup, delivery]",$$,"{'address1': '36-08 Ditmars Blvd', 'address2':...",+17182784800,(718) 278-4800,10527.797867
999,vpkTNjq9dRh9rT3Nh0pe-A,nolita-pizza-new-york-2,Nolita Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/_BKVVo...,False,https://www.yelp.com/biz/nolita-pizza-new-york...,210,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.7208883, 'longitude': -73.9962...","[pickup, delivery]",$,"{'address1': '68 Kenmare St', 'address2': None...",+16468959131,(646) 895-9131,1725.054309


In [47]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

14

In [50]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [53]:
# Verify changes on final DF
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 986 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             986 non-null    object 
 1   alias          986 non-null    object 
 2   name           986 non-null    object 
 3   image_url      986 non-null    object 
 4   is_closed      986 non-null    bool   
 5   url            986 non-null    object 
 6   review_count   986 non-null    int64  
 7   categories     986 non-null    object 
 8   rating         986 non-null    float64
 9   coordinates    986 non-null    object 
 10  transactions   986 non-null    object 
 11  price          793 non-null    object 
 12  location       986 non-null    object 
 13  phone          986 non-null    object 
 14  display_phone  986 non-null    object 
 15  distance       986 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 124.2+ KB
